In [1]:
import nltk
import pickle
import numpy as np
import xml.etree.ElementTree as ET
from gensim.models import word2vec
from nltk.corpus import stopwords
from sklearn import preprocessing

Using gpu device 0: GeForce GT 730M (CNMeM is disabled, cuDNN 5103)


In [2]:
stops = set(stopwords.words("english"))
# # Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 1    # Minimum word count. If a word occurs atleast this number of times it is kept else it is removed.
num_workers = 4       # Number of threads to run in parallel
context = 5           # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

In [3]:
tree = ET.parse("/home/jeet/Academics/CS671/Project/Restaurants_Train.xml")
corpus = tree.getroot()
s = [] # List of list of word tokens.
sentences = corpus.findall('.//sentence')
for sent in sentences:
    tokens = nltk.word_tokenize(sent.find('text').text)
    s.append(tokens)

In [4]:
model = word2vec.Word2Vec(s, workers=num_workers, size=num_features, min_count = min_word_count, \
                          window = context, sample = downsampling)

model.init_sims(replace=True)

# model_name = "300features_1minwords_10context"
# model.save("my_name_w2v_withstopwords")
# model = word2vec.Word2Vec()
# model = word2vec.Word2Vec().load("my_name_w2v_withstopwords")


In [5]:
model = model.load("my_name_w2v_withstopwords")
le = preprocessing.LabelEncoder()
tags = ["CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNS", "NNP", "NNPS", "PDT", "POS", "PRP", "RB", "RBR", "RBS",
        "RP", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WRB", "PRP$", "WP$", ":", "SYM", "$", "--", ".", ",", "(", ")", "''",
        "#"]
#
le.fit(tags)
# #
train_inp = []
train_out = []
i=0

sentences = corpus.findall('.//sentence')
for sent in sentences:
    w = []
    s = nltk.word_tokenize(sent.find('text').text)
    tags_for_sent = nltk.pos_tag(s)
    ind = [0] * len(s)
    i=0
    
    for t in s:
        ohe = [0]*45
        ohe[le.transform(tags_for_sent[i][1])] = 1
        w.append(np.concatenate([model[t], ohe]))
        i = i+1

    train_inp.append(w)
    aspectTerms = sent.find('aspectTerms')
    if (aspectTerms):
        aspectTerm = aspectTerms.findall('aspectTerm')
        if (aspectTerm):
            for aspect_term in aspectTerm:
                try:
                    ind[s.index(aspect_term.attrib['term'])] = 1
                except:
                    continue
    train_out.append(ind)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:29: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.


In [6]:
print np.array(train_inp).shape

(3044,)


Train on 2739 samples, validate on 305 samples
Epoch 1/20
2739/2739 [==============================] - 7s - loss: 0.0104 - acc: 0.1672 - val_loss: 0.0098 - val_acc: 0.1410
Epoch 2/20
2739/2739 [==============================] - 4s - loss: 0.0097 - acc: 0.2881 - val_loss: 0.0095 - val_acc: 0.2426
Epoch 3/20
2739/2739 [==============================] - 4s - loss: 0.0093 - acc: 0.2800 - val_loss: 0.0093 - val_acc: 0.2721
Epoch 4/20
2739/2739 [==============================] - 4s - loss: 0.0090 - acc: 0.2760 - val_loss: 0.0091 - val_acc: 0.2590
Epoch 5/20
2739/2739 [==============================] - 4s - loss: 0.0086 - acc: 0.2946 - val_loss: 0.0095 - val_acc: 0.2852
Epoch 6/20
2739/2739 [==============================] - 4s - loss: 0.0083 - acc: 0.3081 - val_loss: 0.0093 - val_acc: 0.2721
Epoch 7/20
2739/2739 [==============================] - 4s - loss: 0.0079 - acc: 0.3286 - val_loss: 0.0093 - val_acc: 0.2361
Epoch 8/20
2739/2739 [==============================] - 4s - loss: 0.0075 - ac